In [ ]:

import numpy as np 
import pandas as pd


movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

movies.head()

credits.head(2)

movies = movies.merge(credits, on = 'title')

movies.head(2)

# genres , id , keywordds , 

 movies = movies[[ 'movie_id', 'genres', 'keywords', 'overview', 'title', 'cast', 'crew' ]]

movies.head()

movies.isnull().sum()

movies.dropna(inplace = True)

movies.duplicated().sum()

movies.iloc[0].genres

import ast
def convert(obj) :
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L
        

convert('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

movies['genres'] = movies['genres'].apply(convert)

movies.head()

movies['keywords'] = movies['keywords'].apply(convert)

movies.head()


import ast
def convert_cast(obj) :
    L = []
    counter= 0
    for i in ast.literal_eval(obj):
        if counter != 3 :
            L.append(i['name'])
            counter+=1
        else :
            break
    return L

movies['cast'] = movies['cast'].apply(convert_cast)

movies.head()

movies['crew'][0]

import ast
def convert_crew(obj) :
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director' :
            L.append(i['name'])
            break;
    return L

movies['crew'] = movies['crew'].apply(convert_crew)

movies.head()

movies['overview'] = movies['overview'].apply(lambda x:x.split())

movies.head()

movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" " , "") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" " , "") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" " , "") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" " , "") for i in x])

movies.head()

movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

movies.head()

new_df = movies[['movie_id' , 'title' ,  'tags']]

new_df

new_df['tags'] = new_df['tags'].apply(lambda x : " ".join(x))# convert list to string

new_df['tags'][0]

new_df['tags'] = new_df['tags'].apply(lambda x : x.lower())# convert lowercase

new_df['tags'][0]

import nltk
from nltk.stem import PorterStemmer
ps = PorterStemmer()

def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y) 
    

new_df['tags'] = new_df['tags'].apply(stem)

new_df['tags']

from sklearn.feature_extraction.text import CountVectorizer 
cv = CountVectorizer(max_features = 1000 , stop_words = 'english')

vectors = cv.fit_transform(new_df['tags']).toarray()

np.set_printoptions(threshold=np.inf)
print(cv.get_feature_names_out())

from sklearn.metrics.pairwise import cosine_similarity


batch_size = 100  
num_samples = 1000

# Initialize a zero matrix for storing cosine similarities
similarity_matrix = np.zeros((num_samples, num_samples))

# Compute similarity in batches
for i in range(0, num_samples, batch_size):
    for j in range(0, num_samples, batch_size):
        end_i = min(i + batch_size, num_samples)
        end_j = min(j + batch_size, num_samples)

        # Compute cosine similarity for each batch
        similarity_matrix[i:end_i, j:end_j] = cosine_similarity(
            vectors[i:end_i], vectors[j:end_j]
        )

similarity_matrix.shape

similarity_matrix

similarity_matrix[0]

sorted(similarity_matrix[0], reverse = True)#sorted in reverse order but we lost indexes

list(enumerate(similarity_matrix[0])) #we make a tuple of index and score

sorted(list(enumerate(similarity_matrix[0])) ,reverse = True , key = lambda x:x[1])# sort that tuple based on score

new_df.iloc[0].title

def recommend (movie):
    movie_index =  new_df[new_df['title'] == movie].index[0]
    top_movie_list = sorted(list(enumerate(similarity_matrix[movie_index])) , reverse= True , key = lambda x:x[1])[1:6]
    for i in top_movie_list:
        print(new_df.iloc[i[0]].title )

    return 

recommend('Avatar')

import pickle # we need to pass our data to pycharm

pickle.dump(new_df, open('movie.pkl', 'wb'))#write binary mode

pickle.dump(new_df.to_dict(), open('movie_dict.pkl', 'wb'))

 pickle.dump(similarity_matrix , open('similarity.pkl' , 'wb'))

